# Requirements:
- Define a new type called BankAccount that takes a single instance attribute: initial_balance (defaults to 0)
- This type should support a deposit() and withdraw() methods, which in turn should only support transactions in
positive amounts, i.e. an attempt to deposit or withdraw -2 should be ignored
- Define 3 more specialized types of BankAccount with the following characteristics:
 - Savings: has pay_interest() method which deposits directly into the account when called; interest rate: 0.0035
 - HighInterest: like Savings, but higher interest and with a withdrawal fee. The fee is specified at initialization
and defaults to 5. It is taken from the account's balance on every withdrawal. Interest rate: 0.007
 - LockedIn: like HighInterest, but higher interest without the ability to withdraw on demand. Interest rate: 0.009
- The balance of any of the above accounts should be available by attribute access syntax, e.g. account.balance
- The representation of any of the instances should simply indicate the type of account and the amount contained
within, e.g. A SavingsBankAccount with $100 in it.

In [3]:
class BankAccount:

    def __init__(self, initial_balance=0):
        self.initial_balance = initial_balance

    def deposit(self, amount):
        self._check_amount(amount)
        self.initial_balance += amount
        print(f"Deposited ${amount}.")

    def withdraw(self, amount):
        self._check_amount(amount)
        if amount > self.initial_balance:
            raise ValueError("Not Enough Money On Bank Account")
        self.initial_balance -= amount
        print(f"Withdrew ${amount}.")

    def _check_amount(self, amount):
        """ Check that amount is a positive value """
        if amount < 0:
            raise ValueError("amount must be positive")
            
    def get_mro_name(self):
        """ return mro look up chain """
        mro = type(self).__mro__
        return ''.join([obj.__name__ for obj in mro][:-1])
        
    def __repr__(self):
        cls_name = self.get_mro_name()
        return f"A {cls_name} with ${self.initial_balance} in it."

    @property # read only
    def balance(self):
        return self.initial_balance

In [4]:
class Savings(BankAccount):
    RATE = 0.0035
    
    # no need to call __init__(s) and delegate to parents 
    # if not implement Python will look up the mro chains
    
    def pay_interest(self):
        super().deposit(self.initial_balance*self.RATE)

In [5]:
class HighInterest(Savings):
    RATE = 0.007

    def __init__(self, withdrawal_fee = 5, initial_balance = 0):
        super().__init__(initial_balance) # parent delegation must be done
        self.withdrawal_fee = withdrawal_fee

    def withdraw(self, amount):
        super().withdraw(amount+self.withdrawal_fee)

In [6]:
class LockedIn(Savings):
    RATE = 0.009

    def withdraw(self, amount):
        raise NotImplementedError("Can't make an early withdrawal from a Locked-in Savings acount!")

In [7]:
b = BankAccount(initial_balance=100)
b

A BankAccount with $100 in it.

In [8]:
b.deposit(2)
b

Deposited $2.


A BankAccount with $102 in it.

In [9]:
b.withdraw(70)
b

Withdrew $70.


A BankAccount with $32 in it.

In [10]:
s = Savings(140)
s.pay_interest()

Deposited $0.49.


In [11]:
s

A SavingsBankAccount with $140.49 in it.

In [12]:
Savings.__mro__

(__main__.Savings, __main__.BankAccount, object)

In [13]:
def get_name(obj):
    """ return mro look up chain """
    mro = type(obj).__mro__
    return ''.join([obj.__name__ for obj in mro][:-1])

In [14]:
get_name(s)

'SavingsBankAccount'

In [15]:
hi = HighInterest(withdrawal_fee=3)
hi

A HighInterestSavingsBankAccount with $0 in it.

In [16]:
hi.deposit(140)

Deposited $140.


In [17]:
hi.pay_interest()

Deposited $0.98.


In [18]:
hi.withdraw(0.98)

Withdrew $3.98.


In [19]:
hi

A HighInterestSavingsBankAccount with $137.0 in it.

In [20]:
hi.balance

137.0

In [21]:
try:
    hi.balance = 150
except AttributeError as err:
    print(f"{err}")

can't set attribute


In [22]:
l = LockedIn(1000)

In [23]:
l.pay_interest()

Deposited $9.0.


In [24]:
try:
    l.withdraw(1)
except NotImplementedError as err:
    print(err)

Can't make an early withdrawal from a Locked-in Savings acount!


In [50]:
l

A LockedInSavingsBankAccount with $1009.0 in it.